In [58]:
!pip install graphviz
!pip install pydot

In [67]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
folder_repository = "/Users/harun/Desktop/Uni/4. Semester/Data Mining/DataMining_TeamA"

folder_data = f"{folder_repository}/resources/data_classification"
folder_labels = f"{folder_repository}/resources/data_classification/labels"


In [34]:
data = pd.read_csv(f"{folder_data}/x2.csv")
data_labels = pd.read_json(f"{folder_data}/labels/y2.json")

labels = []
# add labels to dataset
for i in range(0, len(data_labels)):
    labels.append(data_labels.iloc[i].item())

data.iloc[:, -1] = labels
data.to_csv('x2.csv', index=False)

/var/folders/lk/v59w6y8j6853sjnmmcghklvh0000gn/T/ipykernel_18919/1773332542.py:9: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.iloc[:, -1] = labels


In [40]:
data = pd.read_csv(f"{folder_data}/x0_with_labels.csv")

# Split datasets into train and test
X_train, X_test, Y_train, Y_test = train_test_split(data.iloc[:, :-1], data.iloc[:, -1], test_size=0.25, shuffle = True)

# labels are intepretated as ordinal, so the model need to know, that the labels are categories
Y_train_encoded = to_categorical(Y_train)
Y_test_encoded = to_categorical(Y_test)

# Create NN model
model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(30, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train_encoded, epochs=10, batch_size=32)
predictions = model.predict(X_test)


Epoch 1/10
687/687 [==============================] - 1s 1ms/step - loss: 0.7771 - accuracy: 0.8662
Epoch 2/10
687/687 [==============================] - 1s 1ms/step - loss: 0.1809 - accuracy: 0.9732
Epoch 3/10
687/687 [==============================] - 1s 1ms/step - loss: 0.1497 - accuracy: 0.9749
Epoch 4/10
687/687 [==============================] - 1s 1ms/step - loss: 0.1319 - accuracy: 0.9769
Epoch 5/10
687/687 [==============================] - 1s 1ms/step - loss: 0.1142 - accuracy: 0.9787
Epoch 6/10
687/687 [==============================] - 1s 1ms/step - loss: 0.1030 - accuracy: 0.9802
Epoch 7/10
687/687 [==============================] - 1s 1ms/step - loss: 0.0927 - accuracy: 0.9822
Epoch 8/10
687/687 [==============================] - 1s 1ms/step - loss: 0.0842 - accuracy: 0.9838
Epoch 9/10
687/687 [==============================] - 1s 1ms/step - loss: 0.0771 - accuracy: 0.9853
Epoch 10/10
229/229 [==============================] - 0s 819us/step
[[1.6835804e-05 3.9977291e-08 7

In [65]:
# extract neurons with most activation
predicted_classes = []
for prediction in predictions:
    predicted_classes.append(np.argmax(prediction))

accuracy = accuracy_score(Y_test, predicted_classes)
print(accuracy)


0.9811526905217154
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [74]:
# weights of the model
weights = model.get_weights()

# weights of first layer do have the most expressiveness, because it is directly connected to the input data
weights_first_layer = weights[0]

# number of features
num_features = weights_first_layer.shape[0]

# generate feature names
feature_names = [f"Feature {i+1}" for i in range(num_features)]

# compute sum of input layer
feature_importance = np.sum(np.abs(weights_first_layer), axis=1)

# sort list with decreasing importance
sorted_features = sorted(zip(feature_importance, feature_names), reverse=True)

# print
for importance, feature_name in sorted_features:
    print(f"{feature_name}:, Importance: {importance}")

Feature 17:, Importance: 20.707456588745117
Feature 19:, Importance: 19.319744110107422
Feature 9:, Importance: 18.89411163330078
Feature 14:, Importance: 18.468786239624023
Feature 18:, Importance: 18.457622528076172
Feature 12:, Importance: 18.331275939941406
Feature 7:, Importance: 18.175268173217773
Feature 13:, Importance: 18.04515266418457
Feature 8:, Importance: 15.890237808227539
Feature 11:, Importance: 12.719953536987305
Feature 15:, Importance: 12.650253295898438
Feature 22:, Importance: 12.629093170166016
Feature 10:, Importance: 12.565032958984375
Feature 6:, Importance: 12.475776672363281
Feature 24:, Importance: 12.190483093261719
Feature 16:, Importance: 11.98433780670166
Feature 20:, Importance: 11.625144004821777
Feature 4:, Importance: 11.289669036865234
Feature 3:, Importance: 11.124408721923828
Feature 23:, Importance: 10.85730266571045
Feature 2:, Importance: 10.642818450927734
Feature 25:, Importance: 10.520696640014648
Feature 1:, Importance: 9.271180152893066
F